In [19]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math, decimal
import requests
dec = decimal.Decimal
import hvplot.pandas
import krakenex
from pykrakenapi import KrakenAPI
api = krakenex.API()
k = KrakenAPI(api)
import json
FearGreedUrl = requests.get('https://api.alternative.me/fng/?limit=2').json()

In [20]:
from pathlib import Path
csvpath1 = Path("./full_moon.csv")

In [21]:
full_moon = pd.read_csv(csvpath1, parse_dates=True, index_col=' Date', infer_datetime_format=True)
full_moon.reset_index(inplace=True)
full_moon = full_moon.rename(columns = {' Date':'Full Moon'})
full_moon.drop(['Day',' Time'], axis=1, inplace=True)

In [22]:
full_moon['New Moon'] = full_moon['Full Moon'] + timedelta(days=14)
full_moon

,Full Moon,New Moon
0,1900-01-15,1900-01-29
1,1900-02-14,1900-02-28
2,1900-03-16,1900-03-30
3,1900-04-15,1900-04-29
4,1900-05-14,1900-05-28
...,...,...
1863,2050-09-01,2050-09-15
1864,2050-09-30,2050-10-14
1865,2050-10-30,2050-11-13
1866,2050-11-28,2050-12-12


In [23]:
ohlc = k.get_ohlc_data('BTCUSD', interval=1440, ascending = True)
Price = ohlc[0]['Price'] = ohlc[0]['close'].rolling(1).mean()
ohlc[0].drop(['high', 'low', 'time', 'open', 'close', 'vwap', 'count', 'volume'], axis = 1).reset_index()

,dtime,Price
0,2020-05-09,9539.7
1,2020-05-10,8716.5
2,2020-05-11,8570.8
3,2020-05-12,8823.7
4,2020-05-13,9317.1
...,...,...
715,2022-04-24,39467.9
716,2022-04-25,40438.5
717,2022-04-26,38121.7
718,2022-04-27,39249.9


In [24]:
# Create a Moon Data table with the BTC Price data for each Full and New Moon date

full_moon = full_moon.join(ohlc[0]['Price'], on = 'Full Moon', how = 'inner')
full_moon['Full Moon Price'] = full_moon['Price']
full_moon = full_moon.drop(columns=['Price']).reset_index()

full_moon = full_moon.join(ohlc[0]['Price'], on = 'New Moon', how = 'inner')
full_moon['New Moon Price'] = full_moon['Price']
full_moon = full_moon.drop(columns=['Price']).reset_index()

moon_data = full_moon.drop(columns=['index','level_0'])

moon_data.tail()


,Full Moon,New Moon,Full Moon Price,New Moon Price
18,2021-11-19,2021-12-03,58099.6,53678.6
19,2021-12-19,2022-01-02,46688.4,47304.6
20,2022-01-18,2022-02-01,42383.0,38719.9
21,2022-02-16,2022-03-02,43873.1,43922.5
22,2022-03-18,2022-04-01,41775.1,46312.0


In [25]:
def FM_result(x, y):
    if x - y < 0:
        return True
    return False

moon_data['FM Result'] = moon_data.apply(lambda row: FM_result(row['Full Moon Price'], row['New Moon Price']), axis=1)
moon_data['FM Percentage Difference'] = (moon_data['Full Moon Price'] - moon_data['New Moon Price']) / moon_data['Full Moon Price']

def NM_result(x, y):
    if x - y > 0:
        return True
    return False

moon_data['NM Result'] = moon_data.apply(lambda row: NM_result(row['New Moon Price'], row['Full Moon Price']), axis=1)
moon_data['NM Percentage Difference'] = (moon_data['New Moon Price'] - moon_data['Full Moon Price']) / moon_data['New Moon Price']

#for ind in (moon_data.index):
#print(moon_data.iloc[:,2])
#print(moon_data.iloc[1:,3]- moon_data.iloc[:,2])


'''things to accomplish tomorrow. 
1. create new column that shows true if the price difference is positive, false if negative
2. create new column that shows the difference between when first bought and where the price is now (this is the control variable)
3. create new column that shows the difference between column created in step 3. and the price sum of the price difference column (this is the dependent variable)
4. plot the dependent variable against the control variable

5. create new column for the fear and greed data
7. create column for the all time high and low
8. calculate correlation between moon phases and all time high and low then plot 
9. calculate correlation between moon phases and fear and greed data then plot

'''


moon_data.head()

,Full Moon,New Moon,Full Moon Price,New Moon Price,FM Result,FM Percentage Difference,NM Result,NM Percentage Difference
0,2020-06-05,2020-06-19,9617.7,9302.1,False,0.032814,False,-0.033928
1,2020-07-05,2020-07-19,9083.8,9213.7,True,-0.014300,True,0.014099
2,2020-08-03,2020-08-17,11237.3,12301.7,True,-0.094720,True,0.086525
3,2020-09-02,2020-09-16,11396.9,10951.1,False,0.039116,False,-0.040708
4,2020-10-01,2020-10-15,10616.1,11501.0,True,-0.083355,True,0.076941


In [26]:
# Create a New Temporary Data Frame to Finalize the Moon Price Behavior Data

frame = {'Full Moon': moon_data.iloc[:,0], 'New Moon': moon_data.iloc[:,1], 'Full Moon Price': moon_data.iloc[:,2], 'New Moon Price': moon_data.iloc[:,3]} #'New Moon Price': moon_data.iloc[1:,3]}
tmpdf = pd.DataFrame(frame)
tmpdf['Difference']= tmpdf['New Moon Price']-tmpdf['Full Moon Price'] #.shift(1)
tmpdf['Percentage Difference'] = ((tmpdf['Full Moon Price'] - tmpdf['New Moon Price']) / tmpdf['New Moon Price'])*100
tmpdf['Trade Profit']= tmpdf['Difference'].cumsum()
tmpdf['Buy and Hold']= tmpdf['New Moon Price']-tmpdf['Full Moon Price'][0]

def returnBoolFM(x):
    if x < 0:
        return True
    return False

def returnBoolNM(x):
    if x > 0:
        return True
    return False

tmpdf['Did We Profit this Month'] = tmpdf.apply(lambda row: returnBoolNM(row['Difference']), axis = 1)


tmpdf

,Full Moon,New Moon,Full Moon Price,New Moon Price,Difference,Percentage Difference,Trade Profit,Buy and Hold,Did We Profit this Month
0,2020-06-05,2020-06-19,9617.7,9302.1,-315.6,3.392782,-315.6,-315.6,False
1,2020-07-05,2020-07-19,9083.8,9213.7,129.9,-1.409857,-185.7,-404.0,True
2,2020-08-03,2020-08-17,11237.3,12301.7,1064.4,-8.652463,878.7,2684.0,True
3,2020-09-02,2020-09-16,11396.9,10951.1,-445.8,4.070824,432.9,1333.4,False
4,2020-10-01,2020-10-15,10616.1,11501.0,884.9,-7.694114,1317.8,1883.3,True
5,2020-10-31,2020-11-14,13809.1,16080.1,2271.0,-14.123046,3588.8,6462.4,True
6,2020-11-30,2020-12-14,19708.1,19281.1,-427.0,2.214604,3161.8,9663.4,False
7,2020-12-30,2021-01-13,28879.9,37390.1,8510.2,-22.760570,11672.0,27772.4,True
8,2021-01-28,2021-02-11,33430.4,47962.8,14532.4,-30.299315,26204.4,38345.1,True
9,2021-02-27,2021-03-13,46170.2,61174.9,15004.7,-24.527543,41209.1,51557.2,True


In [27]:
# Define a function to count the Trues in the 'Did We Profit this Month' column
def count_tr(x):
    return x.sum()
number_of_Trues = (count_tr(tmpdf['Did We Profit this Month']))

#Count the number of rows in the Did We Profit this Month column

number_of_Rows = tmpdf['Did We Profit this Month'].count()

# The PERCENTAGE of times you would profit if you bought on Full Moon and sold on New Moon each month

print(f'{(number_of_Trues/number_of_Rows)*100:.2f}% is the percentage of times you would profit if you bought BTC on the full moon and sold it on the new moon of every month!')

65.22% is the percentage of times you would profit if you bought BTC on the full moon and sold it on the new moon of every month!


In [28]:
#7. create column for the all time high and low
# moon_data['All Time High'] = Price.max()
# moon_data


In [29]:
# create hvplot figures to then overaly 
glyph_1 = ohlc[0]['Price'].hvplot.line( 
    'dtime', 'Price',
    color='purple',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_2 = moon_data.hvplot.scatter(
    x = 'Full Moon',
    y = 'Full Moon Price',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_3 = moon_data.hvplot.scatter(
    x = 'New Moon',
    y = 'New Moon Price',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)


In [30]:
#overaly the figures created above 
glyph_1*glyph_2*glyph_3

:Overlay
   .Curve.Price :Curve   [dtime]   (Price)
   .Scatter.I   :Scatter   [Full Moon]   (Full Moon Price)
   .Scatter.II  :Scatter   [New Moon]   (New Moon Price)

In [31]:
print(json.dumps(FearGreedUrl["data"][0], indent=4))

{
    "value": "24",
    "value_classification": "Extreme Fear",
    "timestamp": "1651104000",
    "time_until_update": "81117"
}
